In [25]:
import geopandas as gpd
import pandas as pd
import numpy as np
import requests as re
import osmnx as ox
import networkx as nx
import h3
import os
import ast
import tqdm
import shapely as shp
import time
import json
import folium

from shapely.ops import unary_union, transform
from ortools.constraint_solver import pywrapcp, routing_enums_pb2

file_path = '../config/API_KEY.json'
with open(file=file_path,mode= 'r') as f:
    json_data = json.load(f)
API_KEY = json_data['apiKey']

In [26]:
place_name = "Haarlem, Netherlands"
# Get the boundary of the place
gdf = ox.geocode_to_gdf(place_name)
# Get the boundary polygon
haarlem_boundary = gdf.geometry.values[0]

centroid_haarlem = haarlem_boundary.centroid
map_ = folium.Map(location=[centroid_haarlem.y, centroid_haarlem.x],
                 zoom_start=13)
# Add the boundary to the map
folium.GeoJson(
    haarlem_boundary.__geo_interface__,
    style_function=lambda x: {'color': 'blue',
                            'weight': 2,
                            'fillOpacity': 0.1}
).add_to(map_)
map_

In [30]:
# flip coordinate sequence from longitude, latitude to latitude, longitude as this is how the h3 API reads it
def flip(x,y):
    return y,x
haarlem_boundary = transform(flip,haarlem_boundary)
# Convert to plan geojson object
haarlem_boundary_geojson = gpd.GeoSeries([haarlem_boundary])\
    .__geo_interface__['features'][0]['geometry']

# Get h3 cell IDs for cells within the bounding polygon / geojson
res8_haarlem_hex = h3.polyfill(haarlem_boundary_geojson,8)

def func_visualize_hexagons(hexagons, color="red", folium_map=None):
    """
    original source: https://nbviewer.org/github/uber/h3-py-notebooks/blob/master/notebooks/usage.ipynb

    hexagons is a list of hexcluster. Each hexcluster is a list of hexagons. 
    eg. [[hex1, hex2], [hex3, hex4]]
    """
    polylines = []
    lat = []
    lng = []
    for hex in hexagons:
        polygons = h3.h3_set_to_multi_polygon([hex], geo_json=False)
        # Flatten polygons into loops.
        outlines = [loop for polygon in polygons for loop in polygon]
        polyline = [outline + [outline[0]] for outline in outlines][0]
        lat.extend(map(lambda v:v[0],polyline))
        lng.extend(map(lambda v:v[1],polyline))
        polylines.append(polyline)
    
    if folium_map is None:
        m = folium\
            .Map(location=[sum(lat)/len(lat),
                        sum(lng)/len(lng)],
                        zoom_start=12, 
                        tiles='openstreetmap')
    else:
        m = folium_map
    for polyline in polylines:
        my_PolyLine=folium.PolyLine(locations=polyline,weight=2,color=color)
        m.add_child(my_PolyLine)
    return m

map_ = func_visualize_hexagons(hexagons=res8_haarlem_hex)
map_

In [ ]:
res9_haarlem_hex = h3.polyfill(haarlem_boundary_geojson,9)
res10_haarlem_hex = h3.polyfill(haarlem_boundary_geojson,10)

res8_haarlem_hex = pd.DataFrame({'hex_id':list(res8_haarlem_hex),
                                 'hex_resolution':8})
res9_haarlem_hex = pd.DataFrame({'hex_id':list(res9_haarlem_hex),
                                 'hex_resolution':9})
res10_haarlem_hex = pd.DataFrame({'hex_id':list(res10_haarlem_hex),
                                  'hex_resolution':10})
# Concatenate the three dataframes
haarlem_hex_df = pd.concat([res8_haarlem_hex,
                            res9_haarlem_hex,
                            res10_haarlem_hex],
                            ignore_index=True)

# Function to calculate the radius of a hexagon based on
# it's resolution and a buffer value to be safe
def func_get_hex_radius(row,BUFFER):
    edge_length = h3.edge_length(row.hex_resolution,'m')
    radius = edge_length
    radius = radius+edge_length*BUFFER
    return radius

haarlem_hex_df['hex_area'] = haarlem_hex_df\
    .hex_id\
    .apply(lambda x: h3.cell_area(x,unit='m^2'))
haarlem_hex_df['hex_radius_places_api'] = haarlem_hex_df\
    .apply(lambda row: func_get_hex_radius(row,0.15),axis=1)
haarlem_hex_df['centroid'] = haarlem_hex_df\
    .hex_id\
    .apply(lambda x: h3.h3_to_geo(x))

In [ ]:
def func_get_places_poi(lat,lng,resolution,type,api_key):
    
    # Nearby sarch API URL
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    params = {
    'location': f'{lat},{lng}',
    'radius': resolution,
    'types': type,
    'key': api_key}

    response = re.get(url, params=params)
    places_df = []

    results = json.loads(response.content)

    places_df.append(results['results'])

    while 'next_page_token' in results:
        time.sleep(1)
        params['pagetoken'] = results['next_page_token']
        response = re.get(url,params=params)
        results = json.loads(response.content)
        places_df.append(results['results'])
        
    results_df = pd.concat([pd.DataFrame(df) for df in places_df])
    results_df = results_df.reset_index(drop=True)
    
    return results_df

In [ ]:
max_haarlem_hex_def = haarlem_hex_df[haarlem_hex_df.hex_resolution==haarlem_hex_df.hex_resolution.min()]

# All types of POI we want to search for
ALL_TYPES = ['tourist_attraction','cafe','museum','art_gallery','park','restaurant','zoo',
             'art_gallery','theme_park', 'library','church','shopping_mall']

for type in ALL_TYPES:
    # Create a directory for each type if it doesn't exist
    if not os.path.exists(f'../data/h3cell_output_{type}/'):
        os.makedirs(f'../data/h3cell_output_{type}/')

    print(type)

    searched_cells = os.listdir(f'../data/h3cell_output_{type}/')
    # If the directory is empty, we can start from scratch
    if(len(searched_cells)>0):
        searched_cells = set([x.replace('.csv','') for x in searched_cells])

    # Loop through the hexagons and get the POI data
    for ind,row in tqdm.tqdm(max_haarlem_hex_def.iterrows(),
                            total=max_haarlem_hex_def.shape[0]):
        
        # Parsing the highest resolution cells and saving outputs in a csv file
        h3_cell = row.hex_id
        if(h3_cell not in searched_cells):
            lat,lng = row.centroid[0],row.centroid[1]
            resolution = row.hex_radius_places_api
            # Call the Places API wrapper using the the function we wrote earlier
            df = func_get_places_poi(lat=lat,lng=lng,resolution=resolution,type=type,api_key=API_KEY)
            df['hex_id'] = h3_cell
            cell_save_path = f'../data/h3cell_output_{type}/{h3_cell}.csv'
            df.to_csv(cell_save_path,index=False)
            
            # Key part of the algorithm, if the API returns equal or more than 60 results
            if(len(df)>=60):
                cell_children = h3.h3_to_children(df['hex_id'][0])
                child_df = haarlem_hex_df[haarlem_hex_df.hex_id.isin(cell_children)]
                
                # res 7 ~ 1400m
                for ind,row in child_df.iterrows():
                    h3_cell = row.hex_id
                    lat,lng = row.centroid[0],row.centroid[1]
                    resolution = row.hex_radius_places_api
                    df = func_get_places_poi(lat=lat,lng=lng,resolution=resolution,type=type,api_key=API_KEY)
                    df['hex_id'] = h3_cell
                    cell_save_path = f'../data/h3cell_output_{type}/{h3_cell}.csv'
                    df.to_csv(cell_save_path,index=False)

                    # Repeat same logic as above and all other resolutions  
                    if(len(df)>=60):
                        cell_children = h3.h3_to_children(df['hex_id'][0])
                        child_df = haarlem_hex_df[haarlem_hex_df.hex_id.isin(cell_children)]

                        # res 8 ~ 500m
                        for ind,row in child_df.iterrows():
                            h3_cell = row.hex_id
                            lat,lng = row.centroid[0],row.centroid[1]
                            resolution = row.hex_radius_places_api
                            df = func_get_places_poi(lat=lat,lng=lng,resolution=resolution,type=type,api_key=API_KEY)
                            df['hex_id'] = h3_cell
                            cell_save_path = f'../data/h3cell_output_{type}/{h3_cell}.csv'
                            df.to_csv(cell_save_path,index=False)

paths = [] 
# Collect all the paths of the csv files in the directories and concatenate them into a single dataframe
for type in ALL_TYPES:
    paths += [f'../data/h3cell_output_{type}/{x}' for x in os.listdir(f'../data/h3cell_output_{type}/')]
n_df = pd.concat([pd.read_csv(path) for path in paths])

# Save final dataframe to csv and pickle
n_df.to_csv(f'../data/h3cell_output_all.csv',index=False)
n_df.to_pickle(f'../data/h3cell_output_all.pkl')

In [21]:
n_df

,hex_id,business_status,geometry,icon,icon_background_color,icon_mask_base_uri,name,opening_hours,photos,place_id,...,rating,reference,scope,types,user_ratings_total,vicinity,permanently_closed,price_level,lat,lng
0,8819682511fffff,OPERATIONAL,"{'location': {'lat': 52.3956975, 'lng': 4.6654...",https://maps.gstatic.com/mapfiles/place_api/ic...,#13B5C7,https://maps.gstatic.com/mapfiles/place_api/ic...,NZH Transport Museum,{'open_now': False},"[{'height': 282, 'html_attributions': ['<a hre...",ChIJOb3KvoHlxUcR4olr3LE0edA,...,4.6,ChIJOb3KvoHlxUcR4olr3LE0edA,GOOGLE,"['museum', 'tourist_attraction', 'point_of_int...",279.0,"A. Hofmanweg 35, Haarlem",NaN,NaN,52.395697,4.665406
1,8819682517fffff,OPERATIONAL,"{'location': {'lat': 52.3910141, 'lng': 4.6531...",https://maps.gstatic.com/mapfiles/place_api/ic...,#13B5C7,https://maps.gstatic.com/mapfiles/place_api/ic...,Barrel Organ Museum Haarlem,{'open_now': False},"[{'height': 2976, 'html_attributions': ['<a hr...",ChIJjwIPdnzvxUcRr0xJo-uq0Wo,...,4.5,ChIJjwIPdnzvxUcRr0xJo-uq0Wo,GOOGLE,"['museum', 'tourist_attraction', 'point_of_int...",111.0,"Küppersweg 3, Haarlem",NaN,NaN,52.391014,4.653117
2,8819682531fffff,OPERATIONAL,"{'location': {'lat': 52.39047799999999, 'lng':...",https://maps.gstatic.com/mapfiles/place_api/ic...,#13B5C7,https://maps.gstatic.com/mapfiles/place_api/ic...,Het Dolhuys,{'open_now': False},"[{'height': 3120, 'html_attributions': ['<a hr...",ChIJxfcBs3PvxUcReOojnqq1zx4,...,4.0,ChIJxfcBs3PvxUcReOojnqq1zx4,GOOGLE,"['museum', 'tourist_attraction', 'cafe', 'stor...",1080.0,"Schotersingel 2, Haarlem",NaN,NaN,52.390478,4.638058
3,8819682531fffff,OPERATIONAL,"{'location': {'lat': 52.39611109999999, 'lng':...",https://maps.gstatic.com/mapfiles/place_api/ic...,#4DB546,https://maps.gstatic.com/mapfiles/place_api/ic...,Haarlemmer Kweektuin,{'open_now': False},"[{'height': 2988, 'html_attributions': ['<a hr...",ChIJ8UReJHXvxUcR_j0h1eSUV74,...,4.5,ChIJ8UReJHXvxUcR_j0h1eSUV74,GOOGLE,"['park', 'tourist_attraction', 'point_of_inter...",1026.0,"Kleverlaan 9, Haarlem",NaN,NaN,52.396111,4.635833
4,8819682533fffff,OPERATIONAL,"{'location': {'lat': 52.3892523, 'lng': 4.6379...",https://maps.gstatic.com/mapfiles/place_api/ic...,#4DB546,https://maps.gstatic.com/mapfiles/place_api/ic...,De Bolwerken,{'open_now': True},"[{'height': 1600, 'html_attributions': ['<a hr...",ChIJa-SPXXLvxUcR7SESkcnvoJ0,...,4.6,ChIJa-SPXXLvxUcR7SESkcnvoJ0,GOOGLE,"['park', 'tourist_attraction', 'point_of_inter...",476.0,"De Bolwerken, MK, Haarlem",NaN,NaN,52.389252,4.637936
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1394,8819682ed9fffff,OPERATIONAL,"{'location': {'lat': 52.38046970000001, 'lng':...",https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,Stadshartkerk,{'open_now': False},"[{'height': 1600, 'html_attributions': ['<a hr...",ChIJSTx0PADvxUcRVgOGTc_rz_E,...,NaN,ChIJSTx0PADvxUcRVgOGTc_rz_E,GOOGLE,"['church', 'place_of_worship', 'point_of_inter...",NaN,"Gedempte Oude Gracht 61, Haarlem",NaN,NaN,52.380470,4.632959
1395,8819682ed9fffff,OPERATIONAL,"{'location': {'lat': 52.37728, 'lng': 4.629745...",https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,Nieuwe kerk,NaN,"[{'height': 857, 'html_attributions': ['<a hre...",ChIJ_wZpcQDvxUcRGsFo7rKvJEA,...,5.0,ChIJ_wZpcQDvxUcRGsFo7rKvJEA,GOOGLE,"['church', 'place_of_worship', 'point_of_inter...",3.0,"Nieuwe Kerksplein 36, Haarlem",NaN,NaN,52.377280,4.629746
1396,881968253bfffff,OPERATIONAL,"{'location': {'lat': 52.3942076, 'lng': 4.6416...",https://maps.gstatic.com/mapfiles/place_api/ic...,#4B96F3,https://maps.gstatic.com/mapfiles/place_api/ic...,Cronjé Winkelstraat,NaN,"[{'height': 3024, 'html_attributions': ['<a hr...",ChIJLTU0QyXvxUcR8nSjKz5kLk4,...,4.1,ChIJLTU0QyXvxUcR8nSjKz5kLk4,GOOGLE,"['shopping_mall', 'point_of_interest', 'establ...",20.0,"Generaal Cronjéstraat, Haarlem",NaN,NaN,52.394208,4.641687
1397,88196

In [14]:
# read pickled data and convert to GeoDataFrame
n_df = pd.read_pickle(f'../data/h3cell_output_all.pkl')
n_df['geometry'] = n_df['geometry'].apply(lambda x: ast.literal_eval(x))
n_df['lat'] = n_df['geometry'].apply(lambda x: x['location']['lat'])
n_df['lng'] = n_df['geometry'].apply(lambda x: x['location']['lng'])
n_gdf = n_df.copy()
n_gdf['geometry'] = n_gdf['geometry'].apply(lambda row: shp.geometry.Point(row['location']['lat'],row['location']['lng']))
n_gdf = gpd.GeoDataFrame(n_gdf, geometry='geometry', crs='EPSG:4326')

# visualize all POI on a map using folium
map_ = folium.Map(location=[n_gdf.lat.mean(),
                            n_gdf.lng.mean()],
        zoom_start=12,
        tiles='openstreetmap')
for i in range(len(n_gdf)):
    folium.CircleMarker(location=[n_gdf.iloc[i]['lat'], n_gdf.iloc[i]['lng']],
                  popup=n_gdf.iloc[i]['name'],
                  radius=2,
                  fill=True,
                  color='red',
                  weight=1,
                  fill_opacity=1).add_to(map_)
    
map_